# COVID TWEETS ANALYSIS - Download Tweets
## HA Final Project

In [1]:
import tweepy as twitter
%matplotlib inline
import numpy as np
import pandas as pd
from scipy import stats
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns
import re

## Twitter API

In [2]:
# initialize keys
consumer_key = 'p2XdTzvwngaHqrXqGLmeX4vnC'
consumer_secret = 'b3mKFyCSJCkzrrFgaILAzQeD2a9tMrZJG7idSYLtTwEKzPBEjH'
token='52625368-gLtq0u7zFDrLWm4f0i9fXbZhJk4A4O9nxcBwaVxKD'
token_secret='vHYK3RYBBXwZWTvOskMccbOT0CbGsuLm9hGs0RCI7X9Mb'

In [3]:
# Authenticate to Twitter passing key and secret tokens
auth = twitter.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(token, token_secret)

# Create API object 
api = twitter.API(auth, wait_on_rate_limit=True)

## 1. Download Tweets

In [4]:
# check connection, download tweets with COVID19 related hashtags
hashtag = ['#COVID19','#COVID','#coronavirus']
date = '2021-01-01'
tweets = twitter.Cursor(api.search, q=hashtag,lang='en',since=date, tweet_mode='extended').items(10)
for tweet in tweets:
    for i in range(len(tweet.entities.get('hashtags'))):
        print(tweet.entities.get('hashtags')[i]['text'])

Breaking
Denmark
COVID19
UK
COVID
coronavirus
SunilChhetri
COVID19
Covid_19
coronavirus
coronapaspoort
CovidVaccine
COVID19Vaccination
COVID
CoronaVaccine
CoronavirusVaccine
IndianFootball
COVID19
Denmark
coronavirus
COVID
COVID19dk
CovidVaccine
business
debt
SmallBusinesses
smallbusinessowner
SmallBusiness
smallbusiness
insolvency
liquidation
covid
COVID19
coronavirus
directors
SME
SMEs
coronavirus
covid
covid19
face
mask
safety
stayhome
staysafe
stayhome
business
debt
SmallBusinesses
smallbusinessowner
SmallBusiness
smallbusiness
insolvency
liquidation
covid
COVID19
coronavirus
directors
SME
SMEs
Denmark
COVID19
coronavirus
COVID
Love
Peace
JOY
IDWP
mentalhealth
coronavirus
COVID19
COVID
COVID
covid19
coronavirus
testingshortage


In [5]:
# function to fetch hash tags
def hash_tag(tdata):
    tags = ''
    for i in range(len(tdata)):
        tags = tags + str(tdata[i]['text']) + ','
    tags = tags.strip(',')
    return tags

In [6]:
def extract_place(row):
    if row['Place Info']:
        return row['Place Info'].full_name
    else:
        return None

In [7]:
##Complete function which is going to remove @ , RT , HTTPS , New lines , # Tags , Idea is to clean the ##text.
def clean_data(tweet):
    tweet = re.sub("RT @[\w]*:", "", tweet)
    tweet = re.sub("@[\w]*", "", tweet)
    tweet = re.sub("https://[A-Za-z0-9./]", "", tweet)
    tweet = re.sub("\n", "", tweet)
    tweet = re.sub("&amp", "", tweet)
    tweet = re.sub("#", "", tweet)
    tweet = re.sub(r"[^\w]", ' ', tweet )
    return tweet

##### download 50K tweets - Set 1 (3/8)

In [32]:
tweets = twitter.Cursor(api.search, q=['#COVID19'], lang='en',since='2021-01-01').items(50000)
tweets_data = [[tweet.id_str, 
               tweet.text, 
               tweet.created_at,
               tweet.source, 
               tweet.entities['hashtags'],
               tweet.retweet_count, 
               tweet.favorite_count, 
               tweet.user.screen_name, 
               tweet.user.location,
               tweet.user.verified,
               tweet.user.followers_count,
               tweet.user.statuses_count
              ]for tweet in tweets]

tweets_df = pd.DataFrame(data=tweets_data,columns=['Tweet Id', 
                                                 'text', 
                                                 'Tweet Datetime', 
                                                 'Source',
                                                 'tags',
                                                 'Retweets', 
                                                 'Favorites', 
                                                 'Twitter @ Name', 
                                                 'user loc',
                                                 'verified',
                                                 'followers',
                                                 'tweets'])

pd.set_option('max_colwidth',800)

In [33]:
tweets_df.shape

(50000, 12)

In [34]:
#process tweets , to have clean data for hastags
for i in range(len(tweets_df['tags'])):
    tweets_df['tags'][i] = hash_tag(tweets_df['tags'][i])
    
# apply cleaning function
tweets_df['text'] = tweets_df['text'].apply(lambda x: clean_data(x))

C:\Users\willm\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
tweets_df.sample(10)

,Tweet Id,text,Tweet Datetime,Source,tags,Retweets,Favorites,Twitter @ Name,user loc,verified,followers,tweets
38077,1368949221546659843,IWD2021 Director of Public Health for talks about the impact that COVID19 has had on women,2021-03-08 15:38:22,Twitter for iPad,"IWD2021,COVID19",14,0,CHRISNIELD3,"Chesterfield, England",False,301,2201
15742,1368997499596312587,While face masks may not be comfortable they can protect against COVID19 and won t interfere with your efforts to stay,2021-03-08 18:50:13,Twitter for iPhone,COVID19,89,0,DrHubaEvaluator,"North Carolina, USA 919",False,131508,188431
20627,1368985510077636608,My condolences are with the Price family This is a tragic story but one that needs to be shared as we continue to learn,2021-03-08 18:02:34,Twitter for iPhone,,285,0,angiespizelles,"Los Angeles, CA",False,694,61311
29196,1368966943386963968,Appointments are available today for COVID19 vaccine at sites Cashman Ctr and the Las Vegas Convention Ctr,2021-03-08 16:48:48,Twitter for iPhone,"COVID19,vaccine,Vegas",25,0,princessaleale,,False,82,4944
17436,1368992943076425730,On the day schools open in the UK here are some places where the Covid19 7 day average case rate per 100k is goi co UmsP5v3f0h,2021-03-08 18:32:06,Twitter Web App,"schools,Covid19",0,0,Faewik,London,False,343,1313
32990,1368960156634071040,Reflecting on the legacy of COVID19 COO Amanda Pritchard says we must not think about going backwards co 5CcF07w4KU,2021-03-08 16:21:50,Twitter Web App,COVID19,2,7,NHSAccelerator,"England, United Kingdom",False,11322,7383
23394,1368978977394749447,New CDC recommendations Fully vaccinated people can gather together indoors If grandparents have been vaccinated t,2021-03-08 17:36:37,Twitter Web App,,338,0,jimenez_mej,,False,141,22025
10808,1369011176701624331,Tomorrow at 6 30 AM I m joining to discuss the complete lack of leadership Florida has seen during,2021-03-08 19:44:34,Twitter for iPad,,544,0,Mike281mike,Chicago Illinois,False,1622,46784
2548,1369035007528960000,COVID 19 Update from the CDC March 08 2021 16 15 04 co KC4vL0ajWoCOVID19 COVID19BenHillGeorgia co PqyEhdPZ5c,2021-03-08 21:19:15,Zapier.com,"COVID19,COVID19BenHillGeorgia",0,0,FitzGA_Events,"Fitzgerald, GA",False,14,6661
48270,1368924926753734661,She gave my dad life and then he gave me life and now I m helping her to survive 98 year old Holocaust survivor and,2021-03-08 14:01:50,Twitter for Android,,169,0,LINCSocietyBC,"Mission, British Columbia",False,992,13795


In [36]:
# save to csv
tweets_df.to_csv('COVID19Tweets_HA_final.csv')

In [28]:
tweet_df = pd.read_csv('COVID19Tweets_HA_final.csv')
tweet_df = tweet_df.iloc[:,1:]

In [31]:
tweet_df = tweet_df.drop_duplicates(subset='text', keep="first")
tweet_df.shape

(21479, 12)

##### download 50K tweets - set 2 (3/11)

In [ ]:
tweets2 = twitter.Cursor(api.search, q=['#COVID19'], lang='en',since='2021-01-01').items(25000)
tweets_data2 = [[tweet.id_str, 
               tweet.text, 
               tweet.created_at,
               tweet.source, 
               tweet.entities['hashtags'],
               tweet.retweet_count, 
               tweet.favorite_count, 
               tweet.user.screen_name, 
               tweet.user.location,
               tweet.user.verified,
               tweet.user.followers_count,
               tweet.user.statuses_count
              ]for tweet in tweets2]

In [23]:
tweets_df2 = pd.DataFrame(data=tweets_data2,columns=['Tweet Id', 
                                                 'text', 
                                                 'Tweet Datetime', 
                                                 'Source',
                                                 'tags',
                                                 'Retweets', 
                                                 'Favorites', 
                                                 'Twitter @ Name', 
                                                 'user loc',
                                                 'verified',
                                                 'followers',
                                                 'tweets'])

pd.set_option('max_colwidth',800)

In [24]:
tweets_df2.shape

(25000, 12)

In [25]:
#process tweets , to have clean data for hastags
for i in range(len(tweets_df2['tags'])):
    tweets_df2['tags'][i] = hash_tag(tweets_df2['tags'][i])
    
# apply cleaning function
tweets_df2['text'] = tweets_df2['text'].apply(lambda x: clean_data(x))

C:\Users\willm\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
tweets_df2.head(5)

,Tweet Id,text,Tweet Datetime,Source,tags,Retweets,Favorites,Twitter @ Name,user loc,verified,followers,tweets
0,1370104119097126912,Are you looking for a logo Order on Fiverr co sKkg8JFFp35SOSSpaces notallmen MUNACM thursdayvibes co YBbB6AZEJT,2021-03-11 20:07:31,Twitter Web App,"5SOSSpaces,notallmen,MUNACM,thursdayvibes",0,0,logozaminul,,False,9,284
1,1370104109743869962,The American Rescue Plan provides direct relief to help people across America remain stably housed during COVID19 including,2021-03-11 20:07:29,Twitter for iPad,COVID19,66,0,KerryMcCormack1,"Ohio City, Cleveland",False,5369,3939
2,1370104109269925891,Today has been recognized as National Day of Observance of COVID 19 We re taking the time to reflect on the past y co Q3D8VDynWp,2021-03-11 20:07:29,Hootsuite Inc.,,0,0,MacLeodLorway,Nova Scotia,False,601,2925
3,1370104108611477509,I think we can say that Italy in its third wave New daily COVID19 cases trending up since mid Feb and today reached 25 673,2021-03-11 20:07:29,Twitter for iPhone,"Italy,COVID19",14,0,MiisKali,,False,378,10263
4,1370104106946289668,Today s COVID19_WI update Another day w no deaths to report and good vaccination news 1 in 3 Wisconsinites 65 older are,2021-03-11 20:07:29,Twitter for Android,COVID19_WI,5,0,1SweetLadyBlue,Brewers Country!,False,7491,203582


In [27]:
tweets_df2 = tweets_df2.drop_duplicates(subset='text', keep="first")

In [37]:
# load csv
tweet_df = pd.read_csv('COVID19Tweets_HA_final.csv')
tweet_df = tweet_df.iloc[:,1:]
print(tweet_df.shape)
tweet_df.sample(10)

(50000, 12)


,Tweet Id,text,Tweet Datetime,Source,tags,Retweets,Favorites,Twitter @ Name,user loc,verified,followers,tweets
45563,1368931353077702660,ICYMI I sat down with this weekend to discuss the one year anniversary of COVID19 co sjf9YCIL9j,2021-03-08 14:27:22,Twitter Web App,COVID19,0,3,LtGovOliver,"New Jersey, USA",True,21493,4871
3827,1369030976316317703,Increasing ventilation is an important prevention strategy for schools and child care programs to reduce the spread of COVID19,2021-03-08 21:03:14,Twitter for iPhone,COVID19,283,0,lucianaroccaaa,Brazil,False,56,3552
33341,1368959598036680716,Be wary of pandemic misinformation and disinformation we can help you learn how to identify avoid fake COVID19 health,2021-03-08 16:19:36,Twitter Web App,COVID19,4,0,jpjmolloy,"Dublin City, Ireland",False,888,4183
40411,1368943840468611082,4ELEGANTLY style love usa paris fashion store london luxury beauty madrid covid19 kids clothes shose co 24rY7AK155,2021-03-08 15:16:59,SocialRabbit Plugin,"style,love,usa,paris,fashion,store,london,luxury,beauty,madrid,covid19,kids,clothes,shose",0,0,4elegantly,NaN,False,20,245
8620,1369017031627014149,One year later How do you think the pandemic has changed the retail fashion world COVID19 fashion co wUYCXyqpnw,2021-03-08 20:07:50,Twitter Web App,"COVID19,fashion",0,0,Brandinity,"Hertfordshire, England",False,15267,3379
7626,1369020091891912708,COVID19 vaccines are effective at protecting you from getting sick Based on what we know people who have been fully vaccinat,2021-03-08 20:19:59,Twitterrific for iOS,COVID19,1615,0,7MV7,NaN,False,96,8226
36336,1368953413644541984,Check our COVID19 calendar for updated COVID19 testing sites at co dLVASVXvwB Testing is recommended f co EzdurzbQIa,2021-03-08 15:55:02,Buffer,"COVID19,COVID19",0,0,TuSNHD,"Las Vegas, Nevada",False,312,13123
35158,1368955966696357888,On InternationalWomensDay our Vicepresident stresses the need to reflect on the progress made on gender co EYpcEJRpEg,2021-03-08 16:05:11,Twitter Web App,InternationalWomensDay,6,10,EgriseldaL,"Manhattan, NY",False,30831,1467
17925,1368991947440803840,New CDC recommendations Fully vaccinated people can gather together indoors If grandparents have been vaccinated t,2021-03-08 18:28:09,Twitter Web App,NaN,319,0,jadzia_1,USA,False,485,95842
39758,1368945468064010245,People line up at Etobicoke as Toronto enters GreyZone today COVID19 co SKKM1xksvX,2021-03-08 15:23:28,Twitter for iPhone,"Etobicoke,GreyZone,COVID19",0,0,dumboiscrazy,"Toronto, Ontario",False,27,81


In [37]:
# combine with previous file
final_tweet_df = tweet_df.append(tweets_df2).reset_index()
final_tweet_df = final_tweet_df.drop_duplicates(subset='text', keep="first")
final_tweet_df.shape

(34081, 13)

In [38]:
final_tweet_df.nunique()

index             28222
Tweet Id          34081
text              34081
Tweet Datetime    22369
Source              456
tags               8987
Retweets            507
Favorites           218
Twitter @ Name    25082
user loc           8586
verified              2
followers          9806
tweets            18200
dtype: int64

In [39]:
# save to csv
final_tweet_df.to_csv('COVID19Tweets_HA_final_v2.csv')

In [ ]:
# load csv
tweet_df = pd.read_csv('COVID19Tweets_HA_final.csv')
tweet_df = tweet_df.iloc[:,1:]

In [38]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Tweet Id        50000 non-null  int64 
 1   text            50000 non-null  object
 2   Tweet Datetime  50000 non-null  object
 3   Source          49698 non-null  object
 4   tags            37351 non-null  object
 5   Retweets        50000 non-null  int64 
 6   Favorites       50000 non-null  int64 
 7   Twitter @ Name  50000 non-null  object
 8   user loc        36681 non-null  object
 9   verified        50000 non-null  bool  
 10  followers       50000 non-null  int64 
 11  tweets          50000 non-null  int64 
dtypes: bool(1), int64(5), object(6)
memory usage: 4.2+ MB
